In [ ]:
import os
from pathlib import Path

import numpy as np
from PIL import Image
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm

import torch
from facenet_pytorch import MTCNN

from matplotlib import pyplot as plt

In [ ]:
midjourney_root = Path('../data/raw/midjourney')
midjourney_pngs = list(midjourney_root.glob('*.png'))
midjourney_pngs[:5]

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

In [ ]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [ ]:
results = []

for png_path in tqdm(midjourney_pngs):
    img = Image.open(png_path)
    result = mtcnn.detect(img)
    results.append(result)

In [ ]:
results [0]

In [ ]:
def draw_bboxes(image: Image, bboxes: np.array, probabilities: np.array):
    draw = ImageDraw.Draw(image)
    for bbox, prob in zip(bboxes, probabilities):
        draw.rectangle(bbox, fill=None, outline=None, width=1)
        draw.text(bbox[:2], f'{prob: .6f}', anchor='lb')

In [ ]:
img = Image.open(midjourney_pngs[0])

draw_bboxes(img, *results[0])
display(img)

In [ ]:
for png, result in zip(midjourney_pngs, results):
    img = Image.open(png)
    draw_bboxes(img, *result)
    display(img)

In [ ]:
def crop_bboxes(image: Image, bboxes: np.array):
    chips = list(map(lambda bbox: image.crop(bbox), bboxes))
    return chips

In [ ]:
idx = 0
img = Image.open(midjourney_pngs[idx])
bboxes = results[idx][0]

In [ ]:
chips = crop_bboxes(img, bboxes)
for i, chip in enumerate(chips):
    display(chip)
    chip.save(f'chip_{i}.png')

In [ ]:
def plot_histogram(image, color_model=['r', 'g', 'b']):
    w, h = image.size
    pixels = w*h
    channels = image.split()
    for color, channel in zip(color_model, channels):
        hist = list(map(lambda bin: bin/pixels, channel.histogram()))
        plt.plot(hist, color=color)
    plt.xlim([0,256])
    plt.show()

In [ ]:
display(chips[0])
plot_histogram(chips[0])

In [ ]:
chip_cmyk = chips[0].convert('CMYK')
plot_histogram(chip_cmyk, ['c', 'm', 'y'])

In [ ]:
gender_model_path = '../models/Gender_model/final/model_2020-05-02-23-57_accuracy_0.9711_step_38525.pth'